Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [27]:
from itertools import product
from random import random, randint, seed
import numpy as np
from scipy import sparse
from copy import  copy
from functools import reduce

In [28]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [29]:
PROBLEM_SIZE = [100, 1_000, 5_000]
DENSITY = [0.3, 0.7]
PROBABILITY = [0.01]

In [30]:
def fitness(sets, state):
    num_true = np.sum(np.sum(sets[state], axis=0) > 0)
    cost = -sum(state)
    return num_true, cost

In [31]:
def generate_initial_state(prob, problem_size):
    return [True if random() < prob else False for _ in range(problem_size)]

In [ ]:
problems = []
for idx_prob, p in enumerate(PROBLEM_SIZE):
    for d in DENSITY:
        # initial_state = generate_initial_state(idx_prob, p)
        initial_state = [False for _ in range(p)]
        sets = make_set_covering_problem(p, p, d)
        assert fitness([True for _ in range(p)], sets)[0] == p, "Problem not solvable"
        problems.append((sets, initial_state, (p, d)))

In [ ]:
def tweak(state, size):
    new_state = copy(state)
    index = randint(0, size - 1)
    new_state[index] = not state[index]
    return new_state

# Standard Hill Climbing

In [ ]:
def SHC(sets, problem_size, state, max_iteration, limit):
    no_improvment_count = 0
    goodness = fitness(state, sets)
    evaluations = max_iteration + 1
    for i in range(max_iteration):
        if no_improvment_count == limit:
            evaluations = i + 1
            break
        new_state = tweak(state, problem_size)
        goodness_new_state = fitness(new_state, sets)
        if goodness_new_state > goodness:
            state = new_state
            goodness = goodness_new_state
            no_improvment_count = 0
        no_improvment_count += 1
    return state, goodness, evaluations

In [ ]:
max_iteration = 100_000
limit = 50
for sets, initial_state, (p, d) in problems:
    solution_state, goodness, evaluations = SHC(sets, p, initial_state, max_iteration, limit)
    print(
        f"Random Mutation Hill Climbing:\nDensity: {d}\nProblem size: {p}\nNumber of calls to fitness function: {evaluations:,}\nGoodness: {goodness}\n"
    )

# Simulated Annealing Hill Climbing (SAHC)

In [ ]:
def SAHC(sets, problem_size, state, max_iteration, limit):
    no_improvment_count = 0
    goodness = fitness(state, sets)
    evaluations = max_iteration + 1
    for i in range(max_iteration):
        t = 1 - ((i + 1) / max_iteration)
        if t == 0 or no_improvment_count == limit:
            evaluations = i + 1
            break
        new_state = tweak(state, problem_size)
        goodness_new_state = fitness(new_state, sets)
        if goodness_new_state >= goodness or random() < (np.exp(-(goodness[1] - goodness_new_state[1])) / t):
            state = new_state
            goodness = goodness_new_state
            no_improvment_count = 0
        no_improvment_count += 1

    return state, goodness, evaluations

In [ ]:
max_iteration = 5_000
limit = 50
for sets, initial_state, (p, d) in problems:
    solution_state, goodness, evaluations = SAHC(sets, p, initial_state, max_iteration, limit)
    print(
        f"Simulated Annealing:\nDensity: {d}\nProblem size: {p}\nNumber of calls to fitness function: {evaluations:,}\nGoodness: {goodness}\n"
    )